In [ ]:
#Librairies nécessaires
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
#Charge le fichier1

#############################################################
#############################################################
fichier1 = pd.read_excel('data/input_fichier1.xlsx') #Il faudra changer le chemin et/ou le nom du fichier ici d'un mois/trimestre/année à l'autre
#############################################################
#############################################################

fichier1

In [ ]:
#Charge le fichier2
fichier2 = pd.read_excel('data/input_fichier2.xlsx')
fichier2.rename(columns={'Col1':'Nséquence'}, inplace=True)
fichier2['Combi'] = 363
fichier2

In [ ]:
#Charge le fichier3
fichier3 = pd.read_excel('data/input_fichier3.xlsx')
fichier3 = fichier3[:-1]
fichier3['Col1'] = fichier3['Col1'].astype(int)
fichier3.rename(columns={'Col1':'Nséquence', 'Col4':'Deleg'}, inplace=True)
fichier3

In [ ]:
#Charge fichier4 et fichier5
#Elles serviront à créer la colonne 'Tremb'
fichier4 = pd.read_excel('data/input_fichier4.xlsx')
fichier4.rename(columns={'Col1':'Nséquence'}, inplace=True)
fichier4['Freg'] = 'O'
display(fichier4)
fichier5 = pd.read_excel('data/input_fichier5.xlsx')
fichier5.rename(columns={'Col1':'Nséquence'}, inplace=True)
fichier5['Ffra'] = 'O'
display(fichier5)

In [ ]:
#Charge le fichier6
fichier6 = pd.read_csv('data/input_fichier6.csv')
fichier6.rename(columns={'Col2':'Pexecution', 'Col3':'Cregion'}, inplace=True)
fichier6

In [ ]:
#On effectue les jointures
test = (
    fichier1
    .merge(fichier2, on='Nséquence', how='left')
    .merge(fichier3, on='Nséquence', how='left')
    .merge(fichier4, on='Nséquence', how='left')
    .merge(fichier5, on ='Nséquence', how='left')
    .merge(fichier6, on='Pexecution', how='left')
)

test

In [ ]:
#On vérifie pour quels pays le Cregion est vide
test.loc[test['Cregion'].isna()]['Pexecution'].unique()

In [ ]:
#Il ne s'agit que de la France donc on remplace les valeurs manquantes par 0
test['Cregion'] = test['Cregion'].fillna(0).astype(int)
test

In [ ]:
#Liste des actes
liste_actes = ['Liste des actes']

#On ajoute la colonne vide 'Tremb' en avant-dernière position
test.insert(len(test.columns)-1, 'Tremb', None)

#Fonction pour appliquer la logique dans la nouvelle colonne
def assign_type(row):
    if row['Freg'] == 'O' and row['Cacte'] not in liste_actes:
        return 'Régularisation'
    elif row['Ffra'] == 'O' and row['Cacte'] in liste_actes:
        return 'Franchises'
    elif pd.isna(row['Tremb']) and row['Cacte'] not in liste_actes:
        return 'Remboursement'
    else:
        return None  #Si aucune condition n'est remplie, la cellule reste vide
    
#On applique puis on retire les colonnes Flag qui sont désormais inutiles
test['Tremb'] = test.apply(assign_type, axis=1)
test.drop(['Freg', 'Ffra'], axis=1, inplace=True)
test

In [ ]:
#On change le type de la colonne Combi de float à int
test['Combi'] = test['Combi'].astype('Int64')
test = test.iloc[:, -4:]
test

In [ ]:
#Afin d'éviter d'exporter tout le tableau (processus assez long), on n'exporte que les 4 dernières colonnes qu'on pourra copier/coller pour ajouter à la requête (+ ou - 10 secondes)

#############################################################
#############################################################
test.to_excel('data/output_fichier.xlsx', index=False) #Il faudra changer le chemin d'enregistrement d'un trimestre/année à l'autre
#############################################################
#############################################################